In [110]:
from SinsIIStatsThing import getSinsData, createWeaponDict
import json, pprint, glob

In [ ]:
with open(r'Patch 42.07 Localized Text\en.localized_text', 'r') as file:
    LOCALIZED_TEXT = json.load(file)

with open(r'Patch 42.07 Uniforms\gui.uniforms', 'r') as file:
    GUI_UNIFORMS = json.load(file)

ENTITIES_DIRECTORY = r'Patch 42.07 Entities'

subjectGlob = glob.glob(ENTITIES_DIRECTORY + r'\**.research_subject')

entitiesGlob = glob.glob(ENTITIES_DIRECTORY + r'\**')

# with open(r'Patch 41.06 Uniforms\planet.uniforms', 'r') as file:
#     planetUniform = json.load(file)

## Prereqs Search and Print Code

In [112]:
prereqsDict = {}

for i in entitiesGlob:
    entityType = i.split('.')[-1]

    if entityType not in prereqsDict.keys():
        prereqsDict[entityType] = {}

for i in entitiesGlob:
    with open(i, 'r') as file:
        entity = json.load(file)

    entityType = i.split('.')[-1]
    entityString = i.split("\\")[1].split('.')[0]


    if "dlc" not in i:
        race = i.split("\\")[1].split("_")[0]
    else:
        race = i.split("\\")[1].split("_")[1]

    # Create item name
    name = LOCALIZED_TEXT.get('player_race_name.' + race, race)
    try:
        if entityType == 'unit':
            name += f" {LOCALIZED_TEXT[entityString + '_name']}"
        elif entityType == 'ability':
            name += f" {LOCALIZED_TEXT[entityString + '_ability_name']}"
        else:
            name += f" {LOCALIZED_TEXT[entity.get('name')]}"
    except:
        name = entityString

    # Get Prereq
    prerequisiteKeys = {
                        "build_prerequisites",
                        "prerequisites",
                        "level_prerequisites",
                        }
    
    prerequisites = (entity.get(k, '') for k in prerequisiteKeys)

    # prerequisites = entity.get('build_prerequisites', '')
    # if prerequisites == '':
    #     prerequisites = entity.get("prerequisites", '')

    for j in prerequisites:
        if j != '':
            temp = j
            break
        else:
            temp = ''
    prerequisites = temp

    if prerequisites == '':
        try:
            prerequisites = entity["build"]['prerequisites']
        except:
            pass

    if prerequisites != '':
        prereqsDict[entityType][name] = []

        for _, j in enumerate(prerequisites):
            tempList = []
            if len(j) > 0 and type(j[0]) == list:
                for k in j[0]:
                    # Prereq name
                    prerequisiteName = LOCALIZED_TEXT.get('player_race_name.' + race, race)
                    try:
                        prerequisiteName += f" {LOCALIZED_TEXT[k + '_research_subject_name']}"
                    except:
                        prerequisiteName = k
                    tempList.append( prerequisiteName )
            else:
                prerequisiteName = LOCALIZED_TEXT.get('player_race_name.' + race, race)
                try:
                    prerequisiteName += f" {LOCALIZED_TEXT[j[0] + '_research_subject_name']}"
                except:
                    prerequisiteName = j

                if len(prerequisiteName) > 1:
                    tempList.append( prerequisiteName )


            prereqsDict[entityType][name].append( tempList )


In [113]:
pprint.pprint(prereqsDict)

{'ability': {'Advent Culture Deliverance': [[], ['Advent Distant Deliverance']],
             'Advent Dominating Deliverance': [['Advent Dominating '
                                                'Deliverance'],
                                               ['Advent Rapid Deliverance']],
             'Advent Lingering Presence': [['Advent Lingering Presence']],
             'Advent Mend': [['Advent Mend']],
             'Advent Nullify': [['Advent Nullify']],
             'Advent Recharge Antimatter': [[],
                                            ['advent_temple_of_renewal_structure_antimatter_restore_rate']],
             'Advent Recharge Shields': [['Advent Recharge Shields']],
             'Advent Repair': [['Advent Factories Repair']],
             'Advent Repulsion Field': [['Advent Repulsion Field']],
             'Advent Resurrecting Deliverance': [['Advent Resurrecting '
                                                  'Deliverance'],
                                    

In [114]:
with open("wikiprerequisites.json", 'w') as file:
    json.dump(prereqsDict, file, indent = 1)

In [ ]:
# Invert prereqs.json for lookup

prereqsDict2 = dict(prereqsDict)
prereqSearch = {}

for entity, entityDict in prereqsDict2.items():
    for key, item in entityDict.items():
        for i in item:
            if len(i) < 1:
                continue
            elif i[0] not in prereqSearch.keys():
                prereqSearch[i[0]] = []

            if len(item) > 1:
                prereqSearch[i[0]].append( { entity + '_upgrade' : key } )
            else:
                prereqSearch[i[0]].append( { entity : key } )
            

In [116]:
pprint.pprint(prereqSearch)

{'Advent Accumulated Will': [{'research_subject': 'Advent Acclimatization of '
                                                  'Will'},
                             {'research_subject': 'Advent Collective Focus'}],
 'Advent Acolyte Unlock': [{'unit': 'Advent Acolyte Corvette'}],
 'Advent Additional Armor Unlock': [{'unit_item': 'Advent Additional Armor'}],
 'Advent Adept Psi Training': [{'research_subject': 'Advent Essence Reservoir '
                                                    'Unlock'},
                               {'research_subject': 'Advent Brilliance Array '
                                                    'Unlock'},
                               {'research_subject': 'Advent Consecration '
                                                    'Inducer Unlock'},
                               {'research_subject': 'Advent Vex Amplifier '
                                                    'Unlock'},
                               {'research_subject': 'Advent Master Ps

## WikiResearch Code

In [119]:
subjectDict = {}

def getTierNumerals(tier):
        tier = str(tier)

        tierDict = {'0' : 'I',
                    '1' : 'II',
                    '2' : 'III',
                    '3' : 'IV',
                    '4' : 'V'
                    }
        return tierDict[tier]

def unpackPrice(price : dict):
    price = (price.get('credits', 0), price.get('metal', 0), price.get('crystal', 0))
    
    # 'price': {'credits': 850.0,
    #             'crystal': 650.0,
    #             'metal': 275.0},
    return price

def getModifierName(modifierID, modifierType):
    # modifierClasses = ["unit_modifiers",
    #                 "weapon_modifiers",
    #                 "planet_modifiers",
    #                 "empire_modifiers",
    #                 "npc_modifiers",
    #                 "unit_factory_modifiers",
    #                 "exotic_factory_modifiers"]

    # if modifierType == "unit_factory_modifiers":
    #     return LOCALIZED_TEXT.get(f"unit_factory_modifier.{modifierID}", modifierID)
    
    # if modifierType == "exotic_factory_modifiers":
    #     return LOCALIZED_TEXT.get(f"exotic_factory_modifier.{modifierID}", modifierID)
    
    # if modifierType == "npc_modifiers":
    #     return LOCALIZED_TEXT.get(f"npc_modifier.{modifierID}", modifierID)
    
    if modifierType == "planet_modifiers":
        return LOCALIZED_TEXT.get(GUI_UNIFORMS["modifier"]["planet_modifier_names"].get(modifierID), modifierID)
    
    return LOCALIZED_TEXT.get( f"unit_modifier.{modifierID}",
                        LOCALIZED_TEXT.get(f"empire_modifier.{modifierID}",
                        LOCALIZED_TEXT.get(f"weapon_modifier.{modifierID}",
                        LOCALIZED_TEXT.get(f"planet_modifier.{modifierID}",
                        LOCALIZED_TEXT.get(f"npc_modifier.{modifierID}",
                        LOCALIZED_TEXT.get(f"unit_factory_modifier.{modifierID}",
                        LOCALIZED_TEXT.get(f"exotic_factory_modifier.{modifierID}",
                        modifierID )))))))
     
    

def formatNumberValue(number):
    if str(number)[-1] == "0":
        number = int(number)
    return abs(number)

def getTags(modifier):

    tagTypeList = ['tags',
                   'planet_types',
                   ]

    for tagType in tagTypeList:
        tags = modifier.get(tagType, '')

        if tags != '':
            break

    if tags != '':
        temp = ""
        for i, tag in enumerate(tags):
            if i < len(tags) - 1: 
                temp = temp.replace(" Planet", "") + f"{LOCALIZED_TEXT.get(f"{tag}_planet_name" , LOCALIZED_TEXT.get(f"weapon_tag_name.{tag}" , tag.capitalize()))}, "
            elif i == len(tags) - 1 and i == 0:
                temp = f"{LOCALIZED_TEXT.get(f"{tag}_planet_name" , LOCALIZED_TEXT.get(f"weapon_tag_name.{tag}" , tag.capitalize()))}"
            elif i == len(tags) - 1 and i == 1:
                temp = temp[:-2].replace(" Planet", "") + f" and {LOCALIZED_TEXT.get(f"{tag}_planet_name" , LOCALIZED_TEXT.get(f"weapon_tag_name.{tag}" , tag.capitalize()))}"
            else:
                temp = temp.replace(" Planet", "") + f"and {LOCALIZED_TEXT.get(f"{tag}_planet_name" , LOCALIZED_TEXT.get(f"weapon_tag_name.{tag}" , tag.capitalize()))}"
        tags = temp
    else:
        tags = False

    return tags

# ----------------------------------------------------------------------------------------------

for i in subjectGlob:
    with open(i, 'r') as file:
        subject = json.load(file)

    # Get Basic Info
    nameCall = subject.pop('name')
    name = LOCALIZED_TEXT.get(nameCall)
    subject['name'] = name

    subject['id'] = i.split('\\')[-1].split('.')[0]

    subject['field'] = subject['field'].split('_')[1].capitalize()

    if name == None:
        print(f"{i} has no name")
        name = i.split("\\")[-1].replace(".research_subject", "")

    if 'vasari' in nameCall:
        race = 'Vasari'
    elif 'trader' in nameCall:
        race = 'TEC'
    elif 'advent' in nameCall:
        race = 'Advent'

    if race:
        name = race + ' ' + name
        subject['race'] = race

    subject['domain'] = LOCALIZED_TEXT.get(f"{race.lower()}_research_domain_partial_name_{subject['domain']}", subject['domain'].capitalize())
    subject['tier'] = f"{subject['domain'].capitalize()} {getTierNumerals(subject['tier'])}" # We're going to change this back like immediately -_-


    try:
        subject['description'] = LOCALIZED_TEXT.get(subject['description'], None)
    except:
        pass

    # Unpack price for some reason

    subject['credits'], subject['metal'], subject['crystal'] = unpackPrice(subject.pop('price', {} ))

    if subject.get('exotic_price'):
        for j in subject.pop('exotic_price'):
            subject[ LOCALIZED_TEXT.get(f"exotic.{j['exotic_type']}.name").lower() ] = j['count']
    

    # Process Prereqs
    try:
        prerequisites = subject['prerequisites']
    except:
        prerequisites = ''

    if prerequisites != '':
        
        subject['prerequisites'] = []

        for _, j in enumerate(prerequisites):
            tempList = []
            if len(j) > 0 and type(j) == list:
                for k in j:
                    # Prereq name
                    prerequisiteName = LOCALIZED_TEXT.get('player_race_name.' + race, race)
                    try:
                        prerequisiteName += f" {LOCALIZED_TEXT[k + '_research_subject_name']}"
                    except:
                        prerequisiteName = k

                    tempList.append(prerequisiteName)
                subject['prerequisites'].append(tempList)

    # Process Modifiers
    modifierClasses = ["unit_modifiers",
                       "weapon_modifiers",
                       "planet_modifiers",
                       "empire_modifiers",
                       "npc_modifiers",
                       "unit_factory_modifiers", ]
                       #"exotic_factory_modifiers"]


    for modType in modifierClasses:
        modifiers = subject.get(modType, False)

        if modifiers:
            for index, modifier in enumerate(modifiers):
                modifierType = getModifierName(modifier.get('modifier_type'), modType)
                behavior = modifier.get('value_behavior')
                value = modifier.get('value')

                tags = getTags(modifier)

                modifiers[index] = f"{"Increases" if value > 0 else "Decreases"} {tags + ' ' if tags else '' }{modifierType} by {f"{formatNumberValue(value*100)}%" if behavior == 'scalar' else formatNumberValue(value)}"

    modifiers = subject.get("exotic_factory_modifiers", False)
    if modifiers:
        for index, modifier in enumerate(modifiers):
            modifierType = getModifierName(modifier.get('modifier_type'), "exotic_factory_modifiers")
            behavior = modifier.get('value_behavior')
            value = modifier.get('value')

            exoticType = modifier.get('exotic_types', '')
            if exoticType != '':
                temp = ""
                for etype in exoticType:
                    temp += f"{LOCALIZED_TEXT.get(f"exotic.{etype}.name")}, "
                exoticType = temp[ : -2] 

            modifiers[index] = f"{"Increases" if value > 0 else "Decreases"} {exoticType} {modifierType} by {f"{formatNumberValue(value*100)}%" if behavior == 'scalar' else formatNumberValue(value)}"

    # Get Unlocks (requires a function below this to run first)
    if prereqSearch.get( name , False ):
        subject["unlocks"] = prereqSearch.get( name )

    # Pop shit we don't want
    subject.pop('name_uppercase', '')
    subject.pop('arbitary_research_line', '')
    subject.pop('version', '')
    subject.pop('extra_text_filter_strings', '')

    # Finally, add processed subject to subject
    subjectDict[name] = subject

pprint.pprint(subjectDict)

Patch 42.07 Entities\trader_shield_burst_restore.research_subject has no name
{'Advent Acclimatization of Will': {'credits': 850.0,
                                    'crystal': 650.0,
                                    'description': 'Prophets of the Unity go '
                                                   'on pilgrimages to planets '
                                                   "under their culture's "
                                                   'influence to lead '
                                                   'believers in meditation, '
                                                   'granting an increase in '
                                                   'maximum focus and focus '
                                                   'regeneration.',
                                    'domain': 'Harmony',
                                    'field': 'Evangelism',
                                    'field_coord': [5, 8],
                             

In [ ]:
with open('wikiresearch.json', 'w') as file:
    json.dump(subjectDict, file, indent = 1)

In [ ]:
fields = []
for _, i in subjectDict.items():
    if i['field'] not in fields:
        fields.append(i['field'])

for j in fields:
    print(j)

Ascension
Temp
Material
Evangelism
Energy
Protection
Reclamation
Experimental
Engineering
Annexation
Armament
Mobilization
Policy
Industry
Survival
Nanotech
Revival
Oppression
Remnants
Phase
Assault


In [ ]:
for i in subjectDict.keys():
    print(f"[[File:SoaSE2_{i.replace(" ", "_")}.png|thumb|{i}]]") #

[[File:SoaSE2_Advent_Ritualized_Instruction.png|thumb|Advent Ritualized Instruction]]
[[File:SoaSE2_Advent_Ascension_Rites.png|thumb|Advent Ascension Rites]]
[[File:SoaSE2_Advent_Reanimation.png|thumb|Advent Reanimation]]
[[File:SoaSE2_Advent_Commerce_Integration.png|thumb|Advent Commerce Integration]]
[[File:SoaSE2_Advent_Commerce_Adaptation.png|thumb|Advent Commerce Adaptation]]
[[File:SoaSE2_Advent_Commerce_Acceptance.png|thumb|Advent Commerce Acceptance]]
[[File:SoaSE2_Advent_Forewarning.png|thumb|Advent Forewarning]]
[[File:SoaSE2_Advent_Acute_Premonitions.png|thumb|Advent Acute Premonitions]]
[[File:SoaSE2_Advent_Andvar_Cultivation.png|thumb|Advent Andvar Cultivation]]
[[File:SoaSE2_Advent_Wall_of_Faith.png|thumb|Advent Wall of Faith]]
[[File:SoaSE2_Advent_Fortress_of_Faith.png|thumb|Advent Fortress of Faith]]
[[File:SoaSE2_Advent_High_Efficiency_Jumps.png|thumb|Advent High Efficiency Jumps]]
[[File:SoaSE2_Advent_Improved_Meteoroid_Control.png|thumb|Advent Improved Meteoroid Cont